In [ ]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=822d29edc78ea6f537818d41f65fc3bcc55722ba91148731a5e9279f2c696717
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [ ]:
import pandas as pd
import openai
import time
from fpdf import FPDF

openai.api_key = ''

## Load the dataset

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/ALGOREX-PH/Day-4-AI-First-Dataset-Live/refs/heads/main/Parcel_Information_Dataset.csv')

In [ ]:
df.sample(5)

,Parcel ID,Sender Name,Sender Address,Sender Contact,Receiver Name,Receiver Address,Receiver Contact,Parcel Weight (kg),Parcel Dimensions (cm),Shipment Date,...,Shipping Cost ($),Shipping Method,Tracking Number,Insurance Value ($),Fragile,Temperature Sensitive,Special Instructions,Delivery Attempt Count,Last Location,Courier ID
4,PCL935942,Alice Green,"101 Pine St, Houston, TX 77001",+1-555-3594,Mike Lee,"101 Pine St, Houston, TX 77001",+1-555-8828,13.20,43x83x93,2024-11-18,...,9.07,Express,TRK897242703,255.49,No,No,Deliver to reception,1,"Distribution Center, Phoenix, AZ",CR9727
10,PCL202754,Chris White,"789 Oak Rd, Chicago, IL 60601",+1-555-6164,Jane Smith,"101 Pine St, Houston, TX 77001",+1-555-6717,6.26,37x23x16,2024-11-14,...,37.22,Express,TRK392814023,275.41,Yes,No,Deliver to reception,2,"Distribution Center, Phoenix, AZ",CR6839
14,PCL345674,John Doe,"202 Birch Blvd, Phoenix, AZ 85001",+1-555-9477,Jane Smith,"789 Oak Rd, Chicago, IL 60601",+1-555-5709,3.99,57x65x69,2024-11-23,...,42.27,Express,TRK264978531,384.46,Yes,Yes,Deliver to reception,2,"Distribution Center, Los Angeles, CA",CR1335
19,PCL437818,Chris White,"101 Pine St, Houston, TX 77001",+1-555-3227,Nancy Wilson,"101 Pine St, Houston, TX 77001",+1-555-8283,12.76,46x87x47,2024-11-29,...,38.23,Standard,TRK734214037,419.78,Yes,No,Deliver to reception,3,"Distribution Center, Phoenix, AZ",CR6547
15,PCL285347,Chris White,"202 Birch Blvd, Phoenix, AZ 85001",+1-555-4081,Nancy Wilson,"789 Oak Rd, Chicago, IL 60601",+1-555-4079,9.23,22x36x47,2024-11-01,...,18.66,Express,TRK185876762,63.90,No,Yes,NaN,1,"Distribution Center, Phoenix, AZ",CR9235


## Understanding the Dataset

In [ ]:
# Get Column Names and Data Types
column_info = df.dtypes
print(column_info, "\n")


Parcel ID                   object
Sender Name                 object
Sender Address              object
Sender Contact              object
Receiver Name               object
Receiver Address            object
Receiver Contact            object
Parcel Weight (kg)         float64
Parcel Dimensions (cm)      object
Shipment Date               object
Estimated Delivery Date     object
Delivery Status             object
Shipping Cost ($)          float64
Shipping Method             object
Tracking Number             object
Insurance Value ($)        float64
Fragile                     object
Temperature Sensitive       object
Special Instructions        object
Delivery Attempt Count       int64
Last Location               object
Courier ID                  object
dtype: object 



In [ ]:
# Identify Categorical and Numerical Features
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=['number']).columns

print("Categorical Features:", categorical_features)
print("Numerical Features:", numerical_features)

Categorical Features: Index(['Parcel ID', 'Sender Name', 'Sender Address', 'Sender Contact',
       'Receiver Name', 'Receiver Address', 'Receiver Contact',
       'Parcel Dimensions (cm)', 'Shipment Date', 'Estimated Delivery Date',
       'Delivery Status', 'Shipping Method', 'Tracking Number', 'Fragile',
       'Temperature Sensitive', 'Special Instructions', 'Last Location',
       'Courier ID'],
      dtype='object')
Numerical Features: Index(['Parcel Weight (kg)', 'Shipping Cost ($)', 'Insurance Value ($)',
       'Delivery Attempt Count'],
      dtype='object')


In [ ]:
# Get Unique Values for Categorical Features with Limitation
def get_unique_values(col, max_display=10):
    uniques = df[col].unique().tolist()
    if len(uniques) > max_display:
        return uniques[:max_display] + ["..."]
    return uniques

unique_values = {col: get_unique_values(col) for col in categorical_cols}

print("Unique Values for Categorical Features:")
for col, uniques in unique_values.items():
    print(f" - {col}: {uniques}")
print()

Unique Values for Categorical Features:
 - Parcel ID: ['PCL398114', 'PCL761243', 'PCL104639', 'PCL768961', 'PCL935942', 'PCL595560', 'PCL409380', 'PCL616284', 'PCL137312', 'PCL418086', '...']
 - Sender Name: ['Michael Brown', 'John Doe', 'Linda Black', 'Alice Green', 'Chris White']
 - Sender Address: ['456 Maple Ave, Los Angeles, CA 90001', '101 Pine St, Houston, TX 77001', '202 Birch Blvd, Phoenix, AZ 85001', '123 Elm St, New York, NY 10001', '789 Oak Rd, Chicago, IL 60601']
 - Sender Contact: ['+1-555-4253', '+1-555-2574', '+1-555-3768', '+1-555-2027', '+1-555-3594', '+1-555-1235', '+1-555-7417', '+1-555-7648', '+1-555-5674', '+1-555-8484', '...']
 - Receiver Name: ['Sara Davis', 'Bob Johnson', 'Mike Lee', 'Jane Smith', 'Nancy Wilson']
 - Receiver Address: ['202 Birch Blvd, Phoenix, AZ 85001', '123 Elm St, New York, NY 10001', '101 Pine St, Houston, TX 77001', '789 Oak Rd, Chicago, IL 60601']
 - Receiver Contact: ['+1-555-8367', '+1-555-3140', '+1-555-9256', '+1-555-3973', '+1-555-88

In [ ]:
# Get Descriptive Statistics for Numerical Features
numerical_stats = df[numerical_features].describe()
print("Descriptive Statistics for Numerical Features:\n", numerical_stats, "\n")

Descriptive Statistics for Numerical Features:
        Parcel Weight (kg)  Shipping Cost ($)  Insurance Value ($)  \
count             20.0000          20.000000            20.000000   
mean               9.7480          26.450500           292.237500   
std                5.3491          13.566969           102.527009   
min                1.6400           5.780000            63.900000   
25%                5.2700          13.760000           223.992500   
50%                9.5150          27.400000           277.725000   
75%               13.2350          38.207500           374.115000   
max               18.1200          45.520000           479.580000   

       Delivery Attempt Count  
count               20.000000  
mean                 1.900000  
std                  0.852242  
min                  1.000000  
25%                  1.000000  
50%                  2.000000  
75%                  3.000000  
max                  3.000000   



## Generating the Template

In [ ]:
def generate_template(df,
                      column_info,
                      categorical_features,
                      numerical_features,
                      unique_values,
                      numerical_stats):

    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types:\n"
    for col, dtype in column_info.items():
        column_summary += f" - {col}: {dtype}\n"

    # Unique values for categorical features
    unique_values_str = "Unique Values for Categorical Features:\n"
    for col, uniques in unique_values.items():
        unique_values_str += f" - {col}: {uniques}\n"

    # Descriptive statistics for numerical features
    numerical_stats_str = "Descriptive Statistics for Numerical Features:\n"
    for col in numerical_features:
        numerical_stats_str += f" - {col}:\n"
        for stat_name, value in numerical_stats[col].items():
            numerical_stats_str += f"   {stat_name}: {value}\n"

    # Define the system prompt
    system_prompt = """
    You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness."""

    # Define the user prompt
    user_prompt = f"""
    Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."

    Output only the template without any explanation or introduction.
    The template's variables will be dynamically replaced so make sure they're formatted properly."""

    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                temperature=0.3,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            template = response['choices'][0]['message']['content'].strip()
            return template
        except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(2)  # Wait before retrying

    return None

In [ ]:
# Generate the template
template = generate_template(df,
                             column_info,
                             categorical_features,
                             numerical_features,
                             unique_values,
                             numerical_stats)
print(template)

"Parcel ID {Parcel ID} was shipped by {Sender Name} from {Sender Address} with contact number {Sender Contact} to {Receiver Name} at {Receiver Address}, reachable at {Receiver Contact}. The parcel weighs {Parcel Weight (kg)} kg and has dimensions of {Parcel Dimensions (cm)}. It was shipped on {Shipment Date} and is estimated to be delivered by {Estimated Delivery Date}. The current delivery status is '{Delivery Status}', and it was sent via {Shipping Method} with a tracking number of {Tracking Number}. The shipping cost was ${Shipping Cost ($)} and the insurance value is ${Insurance Value ($)}. The parcel is marked as {Fragile} and {Temperature Sensitive}. Special instructions include: {Special Instructions}. There have been {Delivery Attempt Count} delivery attempts, and the last known location of the parcel was {Last Location}. The courier ID for this shipment is {Courier ID}."


## Populating the Template with Actual Values

In [ ]:
def populate_template(template, row):

    # Convert row to dictionary and replace NaN with 'N/A'
    row_dict = row.to_dict()
    for key, value in row_dict.items():
        if pd.isna(value):
            row_dict[key] = 'N/A'

    # Generate the populated template per row
    paragraph = template.format(**row_dict)

    return paragraph

In [ ]:
# Apply the populate_template function to each row in the dataframe
df['paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)

In [ ]:
df['paragraph'][0]

'"Parcel ID PCL398114 was shipped by Michael Brown from 456 Maple Ave, Los Angeles, CA 90001 with contact number +1-555-4253 to Sara Davis at 202 Birch Blvd, Phoenix, AZ 85001, reachable at +1-555-8367. The parcel weighs 7.75 kg and has dimensions of 97x16x70. It was shipped on 2024-11-17 and is estimated to be delivered by 2024-11-17. The current delivery status is \'Delivered\', and it was sent via Standard with a tracking number of TRK389255020. The shipping cost was $8.66 and the insurance value is $257.34. The parcel is marked as Yes and No. Special instructions include: N/A. There have been 1 delivery attempts, and the last known location of the parcel was Distribution Center, Phoenix, AZ. The courier ID for this shipment is CR9571."'

## Saving the Populated Templates (Natural Language Texts)

In [ ]:
def save_content_to_txt(paragraphs, txt_filename):
    try:
        with open(txt_filename, 'w', encoding='utf-8') as file:

            for para in paragraphs:
                file.write(para + '\n\n')  # Double newline for separation

        print(f"All content has been saved as '{txt_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to text file: {e}")

In [ ]:
def save_content_to_pdf(paragraphs, pdf_filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)

        # Add each paragraph
        for para in paragraphs:
            pdf.multi_cell(0, 10, para)
            pdf.ln()  # Add a line break between paragraphs

        pdf.output(pdf_filename)
        print(f"All content has been saved as '{pdf_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to PDF: {e}")

In [ ]:
# Save the paragraphs to both text and PDF files
save_content_to_txt(df['paragraph'].to_list(),
                    txt_filename='nlg.txt')

#save_content_to_pdf(df['paragraph'].to_list(),
#                    pdf_filename='nlg.pdf')

All content has been saved as 'nlg.txt'.

